# Prédiction des Éruptions Solaires ----------

**Description :**
* Modèle de machine learning pour prédire les éruptions solaires en utilisant des données solaires historiques. 
* Affectent les communications radio et les réseaux électriques sur Terre. 
    * Parler du blackout de je sais plus quand
    * Y eu un truc récemment
* Prédire ces éruptions peut aider à prendre des mesures préventives pour atténuer leurs impacts.

**Objectifs :**

Mettre en place l'architecture et disposer d'un premier modèle

1. MLFlow, S3 etc.
2. Application 
3. Pour que même si le 1er modèle et les premières prédiction ne sont pas au top, on puisse facilement
    * d'enrichir le dataset
    * faire évoluer le modèle (suivre les performances...)
    * ...

En s'appuyant sur l'architecture en place.

**Technologies et Langages :**
- **Python** : Pour l'extraction des données et le développement des modèles de machine learning.
- **Pandas/NumPy** : Pour la manipulation et l'analyse des données.
- **Scikit-learn/TensorFlow** : Pour la création des modèles de prédiction.
- **NASA Solar Dynamics Observatory (SDO) API** ou **GOES Satellite Data** : Pour collecter des données solaires.
- **MLFlow, AWS...** : Pour l'architecture

**Étapes :**

3 grandes parties:

1. Architecture
1. Modèle
1. Application 

Détails des étapes : 
1. Concevoir et mettre en place l'architecture
1. Collecter des données solaires historiques incluant des événements d'éruption solaire.
1. Prétraiter les données et extraire des caractéristiques pertinentes (activité magnétique, rayonnement, etc.).
1. Construire et entraîner un modèle de machine learning pour prédire les éruptions solaires.
1. Évaluer la performance du modèle et ajuster les hyperparamètres.
1. Déployer une application pour visualiser les prédictions.



## **Sources de Données :**
- **NASA Solar Dynamics Observatory (SDO) API**
- **GOES (Geostationary Operational Environmental Satellite) Satellite Data**
- **Types de Données :**
    - Images du Soleil en différentes longueurs d'onde.
    - Données sur l'activité magnétique solaire (indices de taches solaires, champs magnétiques).
    - Historique des éruptions solaires (classement des éruptions en fonction de leur intensité : classe A, B, C, M, X).


#### Étape 1 : Collecte de Données avec l'API SDO
* collecter des données solaires historiques à partir des API SDO et GOES
* collecter des données sur les taches solaires
* les combiner en un seul DataFrame pour une utilisation ultérieure dans la phase de prétraitement et de modélisation

Nous concentrer sur l'analyse des données et le développement du modèle de machine learning.

In [ ]:
# https://sdo.gsfc.nasa.gov/assets/img/browse/2024/05/01/
# https://sdo.gsfc.nasa.gov/data/bestpractice.php

import requests
import pandas as pd
import datetime as dt

# Fonction pour collecter les données d'images du SDO
def fetch_sdo_data(start_date, end_date, instrument='HMI', wavelength='6173'):
    base_url = 'https://sdo.gsfc.nasa.gov/assets/img/browse/{}/{}/{}/{}.png'
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    images = []
    for date in date_range:
        date_str = date.strftime('%Y/%m/%d')
        image_url = base_url.format(instrument, wavelength, date_str, date_str)
        images.append(image_url)
    
    return images

# Exemple d'utilisation
start_date = '2023-01-01'
end_date = '2023-01-10'
sdo_images = fetch_sdo_data(start_date, end_date)

print("URLs des images SDO :", sdo_images)



#### Étape 2 : Collecte de Données avec l'API GOES

In [ ]:
import requests

# Fonction pour collecter les données des éruptions solaires du GOES
def fetch_goes_data(start_date, end_date):
    base_url = 'https://services.swpc.noaa.gov/json/goes/primary/xrays-1-day.json'
    response = requests.get(base_url)
    
    if response.status_code == 200:
        data = response.json()
        filtered_data = []
        for entry in data:
            timestamp = dt.datetime.strptime(entry['time_tag'], '%Y-%m-%dT%H:%M:%SZ')
            if start_date <= timestamp <= end_date:
                filtered_data.append(entry)
        return filtered_data
    else:
        print("Erreur lors de la récupération des données GOES")
        return []

# Exemple d'utilisation
start_date = dt.datetime(2023, 1, 1)
end_date = dt.datetime(2023, 1, 10)
goes_data = fetch_goes_data(start_date, end_date)

print("Données GOES :", goes_data)


#### Étape 3 : Collecte de Données Historiques sur les Taches Solaires


In [2]:
import requests
import pandas as pd
import datetime as dt

def fetch_sunspot_data(start_date, end_date):
    base_url = 'https://services.swpc.noaa.gov/json/sunspot/observed-sunspot-numbers.json'
    response = requests.get(base_url)
    
    if response.status_code == 200:
        data = response.json()
        sunspot_data = []
        for entry in data:
            date = dt.datetime.strptime(entry['time_tag'], '%Y-%m-%d')
            if start_date <= date <= end_date:
                sunspot_data.append(entry)
        return sunspot_data
    else:
        print("Erreur lors de la récupération des données de taches solaires")
        return []

start_date = dt.datetime(2023, 1, 1)
end_date = dt.datetime(2023, 1, 10)
sunspot_data = fetch_sunspot_data(start_date, end_date)

print("Données de taches solaires :", sunspot_data)


Erreur lors de la récupération des données de taches solaires
Données de taches solaires : []


### Etape 4 : Combinaison et Stockage
Combiner et stocker les données collectées dans un format structuré tel qu'un DataFrame Pandas pour une utilisation ultérieure dans le prétraitement et la modélisation.


In [ ]:
def combine_data(sdo_images, goes_data, sunspot_data):
    # Créer des DataFrames à partir des listes de données
    goes_df = pd.DataFrame(goes_data)
    sunspot_df = pd.DataFrame(sunspot_data)
    
    # Combine les données basées sur les dates
    combined_data = pd.merge(goes_df, sunspot_df, left_on='time_tag', right_on='time_tag', how='outer')
    
    # Ajouter les URLs des images SDO
    combined_data['sdo_image_url'] = pd.Series(sdo_images)
    
    return combined_data

# Exemple de combinaison des données
combined_data = combine_data(sdo_images, goes_data, sunspot_data)
print("Données combinées :", combined_data)



### 2. Prétraitement des Données

**Étapes :**
1. **Nettoyage des Données :** Supprimer ou imputer les valeurs manquantes, filtrer les données bruyantes.
2. **Transformation des Données :** Normaliser les données, transformer les images pour qu'elles aient une taille uniforme.
3. **Annotation des Données :** Associer les éruptions solaires historiques avec les caractéristiques solaires mesurées avant l'éruption.

### 3. Extraction des Caractéristiques

**Caractéristiques Possibles :**
- **Indices de taches solaires** : Nombre de taches solaires et leur distribution.
- **Flux de rayons X** : Mesuré par les satellites GOES.
- **Images de l'atmosphère solaire** : Utilisation de techniques de traitement d'image pour extraire des caractéristiques visuelles.
- **Données magnétiques** : Champs magnétiques mesurés à la surface du Soleil.

### 4. Construction du Modèle

**Choix des Modèles :**
- **Modèles de Régression Logistique** : Pour une approche de base.
- **Random Forests/Gradient Boosting** : Pour la classification et la détection de patterns complexes.
- **Réseaux de Neurones Convolutionnels (CNN)** : Pour l'analyse des images solaires.
- **LSTM (Long Short-Term Memory)** : Pour les données séquentielles et temporelles.

**Entraînement et Validation :**
1. **Séparation des Données** : Diviser les données en ensembles d'entraînement, de validation et de test.
2. **Entraînement des Modèles** : Entraîner différents modèles et comparer leurs performances.
3. **Validation Croisée** : Utiliser la validation croisée pour évaluer la robustesse des modèles.

### 5. Évaluation du Modèle

**Métriques de Performance :**
- **Exactitude (Accuracy)**
- **Précision (Precision)**
- **Rappel (Recall)**
- **F1-Score**

**Analyse des Résultats :**
- Comparer les performances des différents modèles.
- Analyser les erreurs de prédiction et identifier les raisons possibles.

### 6. Déploiement et Visualisation

**Création d'une Application Web :**
- **Flask/Django** : Pour créer une API backend.
- **Dash/Streamlit** : Pour construire une interface utilisateur permettant de visualiser les prédictions en temps réel.

**Fonctionnalités de l'Application :**
- Affichage des prédictions d'éruptions solaires sur un calendrier.
- Visualisation des caractéristiques solaires (indices de taches solaires, flux de rayons X).
- Notifications pour les prévisions d'éruptions imminentes.

### 7. Documentation et Rapport

**Documentation du Code :**
- Explication du processus de collecte de données, de prétraitement, d'extraction de caractéristiques, de modélisation et d'évaluation.

**Rapport de Projet :**
- Présentation des objectifs, de la méthodologie, des résultats et des conclusions.
- Recommandations pour les améliorations futures et les applications pratiques du modèle.

En suivant ce plan, vous pourrer créer un modèle de prédiction des éruptions solaires qui pourra potentiellement être utilisé pour prévenir les impacts des éruptions solaires sur les technologies terrestres.